# 2. Linear Discriminant Model 

In [5]:
def loadDataSet(dataset_path, file_type="txt"):
    if file_type == "txt":
        X = []                                                       ### create feature matrix
        y = []                                                       ### create label matrix
        fr = open(dataset_path)                                      ### open file
        for line in fr.readlines():                                  ### read datum
            lineArr = line.strip().split()                           ### remove the `\n` and obtain the data from string
            X.append([float(x) for x in lineArr[:-1]])               ### add to the feature matrix
            y.append(float(lineArr[-1]))                             ### add to the label matrix
        fr.close()                                                   ### close file
        return X, y 

# read the data
import numpy as np
X_train, y_train = loadDataSet("horseColicTraining.txt")
X_test, y_test = loadDataSet("horseColicTest.txt")

# transform the data from list to np.array
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# normalize
X = np.vstack([X_train, X_test])
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
class LDA(object):
    '''
    This class is for linear discriminant analysis classification.
    
    The class contains the parameters of LDA, including the number of classes and the prior probability p(i) of 
    each class $i$, where $i=1,2,\ldots,num_classes$. Moreover, the class contains the the mean vectors $\mu_i$ 
    and covariance matrix $\Sigma$ of probability distributions $p(x|i)$ for the class $i$.
    
    It also contains the functions for initializing the class, fitting the LDA classifier model, use 
    the fitted model to calculate the linear discriminant functions $\delta_i(x)$ and decision function $h^*(x)$.
    
    Attributes:
        mu (matrix, num_classes*num_features)    : mean vectors of distributions $p(x|i)$. The $i$-th row represents $\mu_i$.
        Sigma (matrix, num_features*num_features): covariance matrix
        num_classes (positive integer)           : the number of classes
        priorProbs (vector, num_classes)         : the prior probability vector and its $i$-th element is $p(i)$
        
    '''
    def __init__(self):
        '''
        Initialize the class by just assigning zero to all atrributes. 
        '''
        self.mu = 0 
        self.Sigma = 0
        self.num_classes = 0
        self.priorProbs = 0
        
    def fit(self, X, y):
        '''
        estimate the mean vector and covariance matrix of each class in the LDA model
        
        Args: 
            X (matrix, num_train*num_features): features of training samples
            y (matrix, num_train): label of training samples
            
        Returns:
            mu (matrix, num_classes*num_features)    : mean vectors of distributions $p(x|i)$. The $i$-th row represents $\mu_i$.
            Sigma (matrix, num_features*num_features): covariance matrix
        ''' 
        num_samples, num_features = X.shape
        values, counts = np.unique(y, return_counts = True)
        num_classes = len(values)
        ### calculate the prior probability $p(i)$
        self.priorProbs = counts / num_samples
        ### calculate the mean vector of each class $\mu_i$
        self.mu = np.zeros((num_classes, num_features))
        for k in range(num_samples):
            self.mu[int(y[k]),:] += X[k,:]
        self.mu = self.mu / np.expand_dims(counts, 1) 
        ### calculate the covariance matrix $\Sigma$
        Sigma_i = [np.cov(X[y == i].T)*(X[y == i].shape[0]-1) for i in range(num_classes)] 
        self.Sigma = sum(Sigma_i) / (X.shape[0]-num_classes)
        return self.mu, self.Sigma
    
    def linear_discriminant_func(self, X):
        '''
        calculate the linear discriminant functions $\delta_i(X)$
        
        Args: 
            X (matrix, num_samples*num_features): features of samples
            
        Returns:
            value (matrix, num_samples*num_classes): the linear discriminant function values. 
            The $(j,i)$-th entry of value represents $\delta_i(X[j,:])$, which is the linear discriminant function value for the class $i$ of the sample at row $j$.
        '''
        ### calculate the inverse matrix of the covariance matrix $\Sigma$
        U, S, V = np.linalg.svd(self.Sigma)
        Sn = np.linalg.inv(np.diag(S))
        Sigma_inv = np.dot(np.dot(V.T, Sn), U.T)
        ### calculate the linear discriminant function values of X
        value = np.dot(np.dot(X, Sigma_inv), self.mu.T) - \
                0.5 * np.multiply(np.dot(self.mu, Sigma_inv).T, self.mu.T).sum(axis = 0).reshape(1, -1) + \
                np.log(np.expand_dims(self.priorProbs, axis = 0))
        return value
    
    def predict(self, X):
        '''
        calculate the linear discriminant functions
        
        Args: 
            X (matrix, num_samples*num_features): features of samples
            
        Returns:
            pred_label (vector, num_samples): the predicted labels of samples. The $j$-th entry represents the predicted label of the sample at row $j$.
        '''
        pred_value = self.linear_discriminant_func(X)
        pred_label = np.argmax(pred_value, axis = 1)
        return pred_label

In [17]:
### initiate the LDA model
model = LDA()
### fit the model with training data and get the estimation of mu and Sigma
mu, Sigma = model.fit(X_train, y_train)
### predict the label of test data
y_pred = model.predict(X_test)
### calculate the accuracy of the fitted LDA model on test data
accuracy = np.sum(y_pred == y_test)/len(y_test)
print("Accuracy of LDA on the test dataset is {}.".format(accuracy))

Accuracy of LDA on the test dataset is 0.7313432835820896.
